In [1]:
import awswrangler as wr
from transformers import pipeline
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
pd.options.mode.copy_on_write = True

In [3]:
df = wr.s3.read_parquet(path="s3://amazon-reviews-eafit/sample/")

In [4]:
df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_temp, test_size=0.5, random_state=42)

In [28]:
def prediction_classifier(
    df: pd.DataFrame,
    size_df: int,
    method="zero-shot-classification",
    model="facebook/bart-large-mnli",
) -> None:
    data = df.head(size_df)
    classifier = pipeline(
        method, model=model, padding="longest", truncation=True, max_length=2048
    )
    if method == "zero-shot-classification":
        results = classifier(
            list(data["review_body"]),
            candidate_labels=["positive", "neutral", "negative"],
        )
        prediction_sentiment = list(map(lambda x: x["labels"][0], results))

    elif method == "sentiment-analysis":
        results = classifier(list(data["review_body"]))
        prediction_sentiment = list(map(lambda x: x["label"].lower(), results))

    else:
        raise NameError("method is not supported")

    data["prediction_sentiment"] = prediction_sentiment

    accuracy = (data["prediction_sentiment"] == data["sentiment"]).sum() / len(
        data["prediction_sentiment"]
    )

    print(f"the accuracy of the model is: {round(accuracy*100,2)}%")

In [29]:
prediction_classifier(
    df_test,
    150,
    method="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The expanded size of the tensor (596) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 596].  Tensor sizes: [1, 514]

In [7]:
prediction_classifier(df_test, 150)

/home/camilo/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


the accuracy of the model is: 85.33%
